In [ ]:
#=
# Package installation
using Pkg
Pkg.add("DataFrames")
Pkg.add("StatsBase")
Pkg.add("NeutralLandscapes")
Pkg.add("PkgCite")
Pkg.add("Plots")
Pkg.add("PGFPlotsX")
Pkg.add("Distributions")
Pkg.add("GeoData")
Pkg.add("Counters")
Pkg.add("ProgressMeter")
Pkg.add("CSV")
=#

In [1]:
using DataFrames
using StatsBase
using Distributions
using Random
using NeutralLandscapes # There's an incompatability compiling this with another package, but the ones here aren't it
using PkgCite
using Plots; pgfplotsx()
using GeoData
using Counters
using ProgressMeter
using CSV

In [36]:
# read Burlington raster 
Buland = geoarray("BurlingtonRaster.gri")
# this raster is ~25km in width

# Get rid of 3rd dim
Buland_array = collect(convert(Array{Float64}, Buland)[:,:,1])

# Check for rows of all NaN
na_sums = sum(convert(Array{Float64}, isnan.(Buland_array)),dims=1)

na_sums[na_sums .== size(Buland_array,2)] #all rows have meaningful values; proceed

# Get percentages of each land cover type
c = Counter{Float64}()
incr!(c, Buland_array)

land_proportions = [c[unique(Buland_array)[i]]/sum(c) for i in 1:length(unique(Buland_array))]

#=
Values from National Land Cover Database
41 = Forest
21 = Developed
81 = Pasture
90 = wetlands
71 = herbaceous
82 = cultivated crops
31 = barren
52 = shrub/scrub
=#

hab_names = ["forest", "developed", "pasture", "wetlands", "herbaceous", "cultivated", "barren", "shrub"]
hab_coefs = [0.12, -0.5, -0.03, 0.57, 0, -0.5, 0, 0.4]
hab_frame = DataFrame(type = hab_names, prop = land_proportions[2:9], coef = hab_coefs)

,type,prop,coef
,String,Float64,Float64
1,forest,0.224461,0.12
2,developed,0.252374,-0.5
3,pasture,0.134121,-0.03
4,wetlands,0.0629525,0.57
5,herbaceous,0.00344292,0.0
6,cultivated,0.0186933,-0.5
7,barren,0.0023558,0.0
8,shrub,0.00209113,0.4


In [37]:
# Create simulated landscapes
land_size = 50 # 1 cell = 0.5 km by 0.5 km

# Create neutral landscape based on real habitat proportions
clustered_land = rand(NearestNeighborCluster(0.2), (land_size, land_size))
landscape = NeutralLandscapes.classify(clustered_land, land_proportions[2:9])

#heatmap(landscape)

50×50 Matrix{Float64}:
 2.0  2.0  2.0  2.0  3.0  3.0  3.0  3.0  …  1.0  1.0  1.0  1.0  3.0  1.0  1.0
 2.0  2.0  2.0  2.0  1.0  1.0  2.0  1.0     1.0  1.0  1.0  1.0  3.0  1.0  1.0
 2.0  2.0  2.0  1.0  1.0  1.0  2.0  2.0     1.0  1.0  1.0  1.0  3.0  1.0  1.0
 2.0  3.0  3.0  1.0  1.0  1.0  2.0  2.0     1.0  1.0  1.0  1.0  3.0  1.0  1.0
 4.0  3.0  1.0  1.0  1.0  3.0  2.0  2.0     2.0  2.0  1.0  1.0  3.0  3.0  1.0
 4.0  1.0  1.0  1.0  3.0  3.0  2.0  2.0  …  2.0  2.0  2.0  2.0  2.0  3.0  3.0
 4.0  1.0  1.0  1.0  3.0  3.0  1.0  1.0     2.0  2.0  2.0  2.0  2.0  3.0  3.0
 4.0  1.0  3.0  3.0  3.0  3.0  1.0  1.0     3.0  2.0  2.0  2.0  2.0  3.0  3.0
 1.0  3.0  3.0  3.0  3.0  3.0  1.0  1.0     3.0  3.0  2.0  2.0  2.0  1.0  1.0
 1.0  3.0  3.0  3.0  3.0  1.0  1.0  1.0     3.0  3.0  4.0  4.0  4.0  1.0  1.0
 ⋮                        ⋮              ⋱            ⋮                   
 4.0  2.0  2.0  1.0  1.0  3.0  2.0  2.0     2.0  2.0  2.0  2.0  4.0  4.0  4.0
 4.0  4.0  2.0  2.0  2.0  2.0  2.0  2.0     

In [38]:
# Define corners of landscape
xmin = 0; xmax = land_size
ymin = 0; ymax = land_size
land_area = xmax*ymax

# Assign density of the lil guys
guy_density = 2.75 # Burlington results had ~11 raccoons per km^2/4 cells per km^2

# get number of guys based on landscape size & density
nguys = rand(Poisson(land_area*guy_density))

# Create data frame of guys
lil_guys = DataFrame(id = string.(collect(1:nguys)), x = convert.(Int64,trunc.(xmax*rand(nguys,1)))[:,1],
    y = convert.(Int64,trunc.(ymax*rand(nguys,1)))[:,1], incubation = 0, time_since_inf = 0, infectious = 0, time_since_disease = 0,
    sex = Int.(rand(Bernoulli(0.5), nguys)), mom = NaN, vaccinated = 0, age = 52)

# Matrix of initial coords for home range attraction
home_coords = DataFrame(id=lil_guys.id, x=lil_guys.x, y=lil_guys.y)

# Initialize disease
lil_guys.incubation[rand(1:nguys,convert(Int64,round(nguys * 0.01)))] .= 1
lil_guys.time_since_inf = ifelse.(lil_guys.incubation .== 1, 1, lil_guys.time_since_inf)

lil_guys[lil_guys.incubation .== 1,:]


,id,x,y,incubation,time_since_inf,infectious,time_since_disease
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,36,42,41,1,1,0,0
2,43,43,38,1,1,0,0
3,90,19,0,1,1,0,0
4,299,4,18,1,1,0,0
5,541,47,22,1,1,0,0
6,571,41,4,1,1,0,0
7,598,37,2,1,1,0,0
8,734,4,36,1,1,0,0
9,1141,10,7,1,1,0,0


In [39]:
# Create function for the guys to look at their surroundings
function look_around(x,y)
    # Get vector of all possible moves
    all_moves = [(x-1, y+1), (x, y+1), (x+1, y+1),
                (x-1, y), (x,y), (x+1, y),
                (x-1, y-1), (x, y-1), (x+1, y-1)]

    # Get indices of impossible moves
    good_moves = findall([(0 .< x[1] .< land_size) .&& (0 .< x[2] .< land_size) for x in all_moves])

    # Remove impossible moves
    poss_moves = copy(all_moves[good_moves])
end

look_around (generic function with 1 method)

In [40]:
# Movement and disease function
function move(coords, dat, reso=500, rate=-0.5)
    # Where coords = list of tuples representing possible moves,
    # dat = data frame of agents,
    # reso = width/height of grid cell in meters
    # rate = rate of distance-decay. Based on trial and error so raccoons typically stay ~1km from home

    # Create blank array
    habs = Vector(undef, length(coords))

    # Get habitat type to create weights
    for i in 1:length(coords)
        habs[i] = landscape[CartesianIndex.([x[1] for x in coords[i]], [x[2] for x in coords[i]])]
    end

    # Match habitats to McClure coefficients
    hab_prefs = Vector(undef, length(coords))
    for i in 1:length(coords)
        hab_prefs[i] = hab_frame.coef[convert.(Int64, habs[i])]
    end
   
    # Movement weights as a function of distance from initial coords
    distances = Vector(undef, length(coords))
    dist_weights = Vector(undef, length(coords))
    for i in 1:length(coords)
        home = (home_coords.x[i], home_coords.y[i])
        distances[i] = ((([x[1] for x in coords[i]].-home[1]).^2 .+ ([x[2] for x in coords[i]].-home[2]).^2)*reso)/100
        dist_weights[i] = exp.(rate .* distances[i])
    end

    # Combine habitat and distance-decay weights
    weights = Vector(undef, length(dist_weights))
    for i in 1:length(dist_weights)
        weights[i] = dist_weights[i] .* hab_prefs[i]
    end

    # Choose new location
    new_location = sample.(coords, Weights.(weights))

    # Make it a df
    new_spots = DataFrame(x = [x[1] for x in new_location], y = [x[2] for x in new_location])

    # Update data frame
    dat.x = deepcopy(new_spots.x)
    dat.y = deepcopy(new_spots.y)

    # Change juvenile coords so they follow mom
    kids = findall(dat.age .< 20)
    
    indices = vcat([findall(dat.id .== dat.mom[kids][i]) for i in 1:length(kids)]...)

    x_coords = dat.x[indices]
    y_coords = dat.y[indices]

    dat.x[kids] = deepcopy(x_coords)
    dat.y[kids] = deepcopy(y_coords)
   
    # Disease transmission
    if length(filter(kv -> kv.second > 1, countmap(new_location))) > 0
        # get coordinates where there are multiple guys
        many_guys = collect(keys(filter(kv -> kv.second > 1, countmap(new_location))))
        indices = [findall(==(x), new_location) for x in many_guys]

        # spread disease
        diseases = deepcopy(dat.infectious)

        for i in 1:length(indices)
            # drop vaccinated guys
            if length(findall(x->x==1, dat.vaccinated[indices[i]])) > 0 & length(indices[i]) > 0
                filter!(x->!(x in findall(x->x==1, dat.vaccinated)), indices[i])
            end

            # spread disease to unvaccinated guys
            if any(diseases[indices[i]] .== 1)
                test = diseases[indices[i]]
                test[test .!= 1].= rand(Bernoulli(0.3), length(test[test .!= 1]))

                diseases[indices[i]] = test
            end  
        end

        # Keep existing infections
        diseases[findall(dat.incubation .== 1)] .= 1
        
        dat.incubation = deepcopy(diseases) 

    end 
end

move (generic function with 3 methods)

In [41]:
# Transition from incubation period to infectious period
function begin_symptoms(dat)
    prob = rand.(Beta.(dat.time_since_inf[(dat.incubation .== 1) .& (dat.infectious .== 0)] .+ 0.000000000001,5))

    if length(prob) > 0
        dat.infectious[(dat.incubation .== 1) .& (dat.infectious .== 0)] = reduce(vcat,rand.(Bernoulli.(prob), 1))
    end
end

begin_symptoms (generic function with 1 method)

In [42]:
# Mortality function
function dont_fear_the_reaper(dat, time=2)
    # disease mortality set at 2 weeks because death function has to occur before movement to avoid bug
    
    # random mortality
    rand_deaths = rand(Bernoulli(0.005),size(lil_guys,1))
    deleteat!(lil_guys, findall(rand_deaths==1))
    deleteat!(home_coords, findall(rand_deaths==1))

    # disease mortality
    deleteat!(home_coords, findall(x -> x > time, dat.time_since_disease))
    filter!(:time_since_disease => <=(time), dat)

    # old age mortality
    deleteat!(home_coords, findall(x -> x >= (52*8), dat.age))
    if length(findall(x-> x>=(52*8), dat.age)) > 0
        filter!(:age => <=(52*8), dat)
    end

    # orphan mortality
    kids = findall(dat.age .< 20)

    no_mom = findall(x -> !(x in dat.id), dat.mom[kids])

    deleteat!(home_coords, findall(x -> !(x in dat.id), dat.mom[kids]))
    filter!(:mom => !in(dat.mom[kids][no_mom]), dat)

    # Density-related mortality
    # get coordinates where there are multiple guys
    new_location = Vector(undef, size(dat,1))
    for i in 1:size(dat,1)
        new_location[i] = (dat.x[i], dat.y[i]) 
    end

    many_guys = collect(keys(filter(kv -> kv.second > 1, countmap(new_location))))
    indices = [findall(==(x), new_location) for x in many_guys]

    # Find cells with max number of guys or greater
    too_many_guys = findall(length.(indices) .> 10) #can adjust this number

    crowded_spots = many_guys[too_many_guys]

    # Apply increased mortality
    crowded_indices = Vector(undef, length(crowded_spots))
    for i in 1:length(crowded_spots)
        crowded_indices[i] = intersect(findall(x -> x == crowded_spots[i][1], lil_guys.x), findall(x -> x == crowded_spots[i][2], lil_guys.y))
    end

    crowding_deaths = sort(unique(vcat(crowded_indices[findall(rand(Bernoulli(0.1), length(crowded_indices)) .== 1)]...)))
    
    
    if length(crowding_deaths) > 0
        deleteat!(dat, crowding_deaths)
        deleteat!(home_coords, crowding_deaths)
    end
end

dont_fear_the_reaper (generic function with 2 methods)

In [43]:
# Reproduction function
function reproduce(dat)
    # Get females only
    females = filter(:sex => ==(1), dat)
    
    # Choose females that actually reproduce
    reproducing = females[randsubseq(1:size(females,1), 0.25),:]

    # Assign number of offspring to each reproducing female
    noffspring = rand(Poisson(2), size(reproducing,1))

    # Create offspring at location of mother
    devil_spawn = DataFrame(x = Int[], y = Int[], mom = String[])
    for i in 1:length(noffspring)
        for j in 1:noffspring[i]
            push!(devil_spawn, (reproducing.x[i], reproducing.y[i], reproducing.id[i]))
        end
    end

    # Fill in missing cols
    devil_spawn.incubation .= 0
    devil_spawn.time_since_inf .= 0
    devil_spawn.infectious .= 0
    devil_spawn.time_since_disease .= 0
    devil_spawn.sex .= Int.(rand(Bernoulli(0.5), size(devil_spawn,1)))
    devil_spawn.vaccinated .= 0
    devil_spawn.age .= 0

    id_vector = collect((maximum(parse.(Int64,dat.id))+1):(maximum(parse.(Int64,dat.id))+size(devil_spawn,1)))
    devil_spawn.id = string.(id_vector)

    # Append to main dataset
    append!(dat, devil_spawn, promote = true)
    append!(home_coords, DataFrame(id = devil_spawn.id, x=devil_spawn.x, y=devil_spawn.y), promote = true)
    
end

reproduce (generic function with 1 method)

In [44]:
# Vaccination function
function ORV(dat, land=nothing, sero_prob=0.2)
    if land==nothing 
        # Create array of vaccination probabilities
        land = fill(sero_prob, (land_size,land_size))
    end

    # Get probs of vaccination at guys' locations
    vaxprob = land[CartesianIndex.(dat.x, dat.y)]

    # Juveniles have lower probs
    kids = findall(dat.age .< 52)
    vaxprob[kids] = vaxprob[kids]./2

    # Generate vaccine outcomes
    vax = Int.(rand.(Bernoulli.(vaxprob)))

    # Update data
    dat.vaccinated = deepcopy(vax)

    dat
end

ORV (generic function with 3 methods)

In [45]:
# Juvenile distribution
function juvies_leave(dat, dispersal_age=30)
    # Get Juveniles
    juvies = dat[findall(x -> x==30, dat.age),:]

    # Create break point so it doesn't get stuck
    niter = 0

    while size(juvies,1) > 0
        niter = niter + 1

        # Pick a direction from list of inline functions
        upleft(x,y)=[x-1, y+1]; up(x,y)=[x, y+1]; upright(x,y)=[x+1, y+1]
        left(x,y)=[x-1, y]; right(x,y)=[x+1, y]
        downleft(x,y)=[x-1, y-1]; down(x,y)=[x, y-1]; downright(x,y)=[x+1, y-1]
        directions = rand([upleft, up, upright, left, right, 
                        downleft, down, downright], size(juvies,1))

        # Get dispersal distance
        distances = rand(Poisson(10), size(juvies,1))

        # RUN!
        coords = Vector(undef, size(juvies,1))

        for i in 1:length(distances)
            coords[i] = [juvies.x[i], juvies.y[i]]
                for j in 1:distances[i]
                coords[i] = directions[i](coords[i][1], coords[i][2])
            end
        end

        # Remove juveniles that left the landscape
        new_juvies = juvies[all.(x-> 0<x<land_size, coords),:]
        gone_juvies = juvies[all.(x-> x<0 || x > land_size, coords),:]

        # Update full data frame
        filter!(:id => !in(gone_juvies.id), dat)
        indices=findall(x-> x in(new_juvies.id), dat.id)
        dat[indices,:]=new_juvies

        # Update home coords data frame
        filter!(:id => !in(gone_juvies.id), home_coords)
        home_coords.x[indices] = new_juvies.x
        home_coords.y[indices] = new_juvies.y
        home_coords.id[indices] = new_juvies.id
        
        # Find coordinates with multiple guys
        new_location = Vector(undef, size(dat,1))
        for i in 1:size(dat,1)
            new_location[i] = (dat.x[i], dat.y[i]) 
        end
    
        many_guys = collect(keys(filter(kv -> kv.second > 1, countmap(new_location))))
        indices = [findall(==(x), new_location) for x in many_guys]
    
        # Find cells with less than max number of guys
        enough_guys = findall(length.(indices) .<= 10) #can adjust this number
    
        good_spots = many_guys[enough_guys]
    
        good_indices = Vector(undef, length(good_spots))
        for i in 1:length(good_spots)
            good_indices[i] = intersect(findall(x -> x == good_spots[i][1], new_juvies.x), findall(x -> x == good_spots[i][2], new_juvies.y))
        end    

        deleteat!(new_juvies,sort(unique(vcat(good_indices...))))

        # Start the cycle again
        juvies = deepcopy(new_juvies)

        if niter > 5
            break
        end
    end
end

juvies_leave (generic function with 2 methods)

In [46]:
# NEXT STEPS: add data frame of summary info

# Create empty data frame
outputs = DataFrame([[], [], [], [], []], ["year", "week", "total_pop", "n_infected", "n_symptomatic"])

#Just the model results
time_steps = 52
years = 3

# progress bar
p = Progress(time_steps*years)

for year in 1:years
    for step in 1:time_steps
        # all guys age 1 week
        lil_guys.age = lil_guys.age .+ 1

        # Juveniles reaching independence (default 30 weeks) disperse
        if size(filter(:age => ==(30), lil_guys),1) > 0 # can change to desired dispersal age
            juvies_leave(lil_guys)
        end

        # Lots of death
        dont_fear_the_reaper(lil_guys, 2)

        # Update time since infection & disease
        lil_guys.time_since_inf[lil_guys.incubation .== 1] = lil_guys.time_since_inf[lil_guys.incubation.==1] .+ 1
        lil_guys.time_since_disease[lil_guys.infectious.==1] = lil_guys.time_since_disease[lil_guys.infectious.==1] .+ 1

        # Reproduction occurs at specific time steps
        if step % 20 == 0
            reproduce(lil_guys)
        end

        # Vaccine baits are distributed at specific time steps
        if step % 45 == 0
            ORV(lil_guys)
        end 

        # Move around
        moves = look_around.(lil_guys.x, lil_guys.y)
        move(moves, lil_guys, 500, -0.05)
        
        # Function where some infected guys become symptomatic
        begin_symptoms(lil_guys)

        # Calculate summary statistics and append to data frame
        row = [years, step, size(lil_guys,1), sum(lil_guys.incubation), sum(lil_guys.infectious)]

        push!(outputs, row)
        
        next!(p)
    end
end
outputs
#lil_guys
#lil_guys[lil_guys.incubation .== 1,:]
CSV.write("outputs.csv", outputs)

Progress:   1%|█                                        |  ETA: 0:03:43

Progress:   2%|█                                        |  ETA: 0:03:08

Progress:   3%|██                                       |  ETA: 0:02:54

Progress:   3%|██                                       |  ETA: 0:02:38

Progress:   4%|██                                       |  ETA: 0:02:28

...

Progress:  26%|███████████                              |  ETA: 0:01:47

Progress:  27%|████████████                             |  ETA: 0:01:51

Progress:  28%|████████████                             |  ETA: 0:01:53

Progress:  28%|████████████                             |  ETA: 0:01:55

BoundsError: BoundsError: attempt to access 50×50 Matrix{Float64} at index [CartesianIndex{2}[CartesianIndex(5, 7), CartesianIndex(19, 22), CartesianIndex(39, 16), CartesianIndex(16, 32), CartesianIndex(5, 15), CartesianIndex(16, 0), CartesianIndex(42, 31), CartesianIndex(48, 3), CartesianIndex(19, 22), CartesianIndex(27, 49), CartesianIndex(33, 43), CartesianIndex(7, 44), CartesianIndex(20, 1), CartesianIndex(9, 27), CartesianIndex(28, 45), CartesianIndex(48, 40), CartesianIndex(9, 23), CartesianIndex(48, 16), CartesianIndex(39, 39), CartesianIndex(40, 26), CartesianIndex(45, 30), CartesianIndex(13, 11), CartesianIndex(0, 17), CartesianIndex(1, 18), CartesianIndex(14, 0), CartesianIndex(43, 20), CartesianIndex(21, 38), CartesianIndex(26, 29), CartesianIndex(3, 44), CartesianIndex(11, 7), CartesianIndex(29, 16), CartesianIndex(0, 22), CartesianIndex(4, 33), CartesianIndex(30, 16), CartesianIndex(36, 15), CartesianIndex(18, 24), CartesianIndex(44, 2), CartesianIndex(7, 2), CartesianInde

In [47]:
#=
#gif
time_steps = 10

anim = @animate for step in 1:time_steps

    moves = look_around.(lil_guys.x, lil_guys.y)
    move(moves, lil_guys)

    fig = Plots.heatmap(landscape, c = :grays)
    Plots.plot!(lil_guys.x, lil_guys.y, seriestype = :scatter, color = lil_guys.infectious)

end

gif(anim, fps = 4)
=#